In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


In [2]:

# Read in the data
tweets_df = pd.read_csv(r"C:\Users\user\PycharmProjects\TwitterYouths\lgbtq_hashtags.csv")
tweets_df.head(10)

,date,id,content,username,likeCount,retweetCount,location,hashtag,source
0,2023-04-12 08:05:43+00:00,1646062024852668418,風の強い日でした\n#セルフポートレート #nana_camera \n#中性俳優 #モデル...,irodor1midori,0,0,NaN,"['セルフポートレート', 'nana_camera', '中性俳優', 'モデル', 'L...","<a href=""http://twitter.com/download/android"" ..."
1,2023-04-12 08:05:34+00:00,1646061984264355841,#Pokekara 友の多重ハモリコーラスで盛り上げてくれています。 #中山美穂『 #色ホワ...,haoyu52,0,0,日本 東京,"['Pokekara', '中山美穂', '色ホワイトブレンド', 'G組合員', 'Gay...","<a href=""http://www.pokekara.com"" rel=""nofollo..."
2,2023-04-12 08:04:49+00:00,1646061799136174082,運動すると頭を空に出来るから気になってたことを整理するきっかけにもなる👏👏\n終わった後の達...,ken19891016,0,0,NaN,"['悩み相談', '悩み聞いて', 'カウンセラー', 'ケアマネ', 'LGBTQ', '...","<a href=""http://twitter.com/download/iphone"" r..."
3,2023-04-12 08:04:28+00:00,1646061709746896897,#gay #LaMasDraga #boy #gaykiss #kep1er #LESSER...,aaasael27,0,0,"Ahome, Sinaloa","['gay', 'LaMasDraga', 'boy', 'gaykiss', 'kep1e...","<a href=""http://twitter.com/download/iphone"" r..."
4,2023-04-12 08:04:04+00:00,1646061607829618688,Sen ancak @RTErdogan'ın tırnağından özür diler...,marjinal_koylu,0,0,NaN,"['LGBT', 'ErdoğanYineKazanacak']","<a href=""http://twitter.com/download/android"" ..."
5,2023-04-12 08:04:02+00:00,1646061599852048385,Hora de descansar estas nalgas jajaja #gay htt...,pepecol69,0,0,"Colima, México",['gay'],"<a href=""http://twitter.com/download/android"" ..."
6,2023-04-12 08:03:58+00:00,1646061584991567873,Pussy! \n\n#LGBTQIA https://t.co/Dr0QpVKr0T,JustChillieh,1,0,"Palm Beach County, FL",['LGBTQIA'],"<a href=""http://twitter.com/download/iphone"" r..."
7,2023-04-12 08:03:04+00:00,1646061355621928960,La Procura di #Padova ha chiesto al Comune gli...,BiribissiBlog,0,0,"Sanremo, Liguria","['Padova', 'gay', 'ANSA']","<a href=""http://twitter.com/download/android"" ..."
8,2023-04-12 08:02:57+00:00,1646061327285116929,Research is key to every #Crypto investment🗣️ ...,RBIF_USA,2,0,"Buffalo, NY","['Crypto', 'DYOR', 'Finance', 'RoboInu', 'Bank...","<a href=""http://twitter.com/download/iphone"" r..."
9,2023-04-12 08:02:54+00:00,1646061316057219073,髪伸びました。\nL/20代/東北\n好きなものは、きなこ餅せんべい\n嫌いなものは信号と横...,izumi_Si3,0,0,NaN,"['LGBTQ', 'LGBTさんと繋がりたい', 'セクマイ', 'セクマイさんと繋がりた...","<a href=""http://twitter.com/download/android"" ..."


In [3]:
tweets_df.drop_duplicates(subset="content",keep="first",inplace=True)

In [4]:
# Remove symbols, numbers, and URL links
tweets_df['content'] = tweets_df['content'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
tweets_df['content'] = tweets_df['content'].apply(lambda x: re.sub(r'\d+', '', x))
tweets_df['content'] = tweets_df['content'].apply(lambda x: re.sub(r'https?://\S+', '', x))
tweets_df['content'] = tweets_df['content'].apply(lambda x: re.sub(r'\d+', '', x))
tweets_df['content'] = tweets_df['content'].apply(lambda x: re.sub(r'#\S+', '', x))
tweets_df["content"]= tweets_df['content'].replace(r'\r', ' ', regex=True)
tweets_df["content"]= tweets_df['content'].replace(r'\n', ' ', regex=True)

In [5]:
tweets_df["content"]

0        風の強い日でした セルフポートレート nana_camera  中性俳優 モデル LGBTQ...
1        Pokekara 友の多重ハモリコーラスで盛り上げてくれています 中山美穂　色ホワイトブレン...
2        運動すると頭を空に出来るから気になってたことを整理するきっかけにもなる 終わった後の達成感な...
3        gay LaMasDraga boy gaykiss keper LESSERAFIM Ne...
4        Sen ancak RTErdoganın tırnağından özür dilersi...
                               ...                        
14997    Guten Morgen ihr lieben  wünsche euch schonmal...
14998    Its all happening in  minutes You all know whe...
14999    Not sure why everyone is so worried about Tran...
15000    elio_vito Libero_official alesallusti Pan per ...
15001    Open VcsBo pijat  gay gayindo GayIndonesia gay...
Name: content, Length: 14691, dtype: object

In [6]:
# Remove emojis
def remove_emojis(content):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', content)

tweets_df['content'] = tweets_df['content'].apply(remove_emojis)

In [7]:
# Remove stop words and lemmatize
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


def process_text(text):
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)


tweets_df['content'] = tweets_df['content'].apply(process_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\user/nltk_data'
    - 'C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python39\\nltk_data'
    - 'C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python39\\share\\nltk_data'
    - 'C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python39\\lib\\nltk_data'
    - 'C:\\Users\\user\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
